In [1]:
from pynq import Overlay
from pynq import MMIO
import pynq.lib.dma
from pynq import allocate
import numpy as np
import struct
from time import time

In [2]:
from pynq import Overlay

overlay = Overlay('/home/xilinx/pynq/overlays/User_overlay/PYNQ_ML.bit')

In [3]:
overlay.is_loaded()

True

In [5]:
from pynq import Clocks

print(f'CPU:   {Clocks.cpu_mhz:.6f}MHz')
print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')
print(f'FCLK1: {Clocks.fclk1_mhz:.6f}MHz')
print(f'FCLK2: {Clocks.fclk2_mhz:.6f}MHz')
print(f'FCLK3: {Clocks.fclk3_mhz:.6f}MHz')

CPU:   650.000000MHz
FCLK0: 100.000000MHz
FCLK1: 100.000000MHz
FCLK2: 100.000000MHz
FCLK3: 100.000000MHz


In [5]:
Clocks.fclk0_mhz = 200

print(f'CPU:   {Clocks.cpu_mhz:.6f}MHz')
print(f'FCLK0: {Clocks.fclk0_mhz:.6f}MHz')
print(f'FCLK1: {Clocks.fclk1_mhz:.6f}MHz')
print(f'FCLK2: {Clocks.fclk2_mhz:.6f}MHz')
print(f'FCLK3: {Clocks.fclk3_mhz:.6f}MHz')

CPU:   650.000000MHz
FCLK0: 200.000000MHz
FCLK1: 100.000000MHz
FCLK2: 100.000000MHz
FCLK3: 100.000000MHz


In [4]:
ip = overlay.NN_inference_axi_0

In [5]:
print("Memory_in_V.address: " + str(ip.register_map.Memory_in_V.address))
print("Memory_out_V.address: " + str(ip.register_map.Memory_out_V.address))

Memory_in_V.address: 32
Memory_out_V.address: 64


In [6]:
in_mmio = MMIO(ip.mmio.base_addr + ip.register_map.Memory_in_V.address, 8 * 4)
ou_mmio = MMIO(ip.mmio.base_addr + ip.register_map.Memory_out_V.address, 3 * 4)

In [138]:
X = np.load('./X_test.npy').astype(np.float32)
X = X[:2]
y = []

In [122]:
def encode(xi):
    return int(round(xi * 2**10))

def encode_ds(xi):
    return np.asarray(np.round(xi*2**10), dtype=np.int)

def encode_pair(xa, xb):
    return encode(xa) + encode(xb) * 2**16
    #return encode(xb) + encode(xa) * 2**16
    
def encode_whole_ds(dataset):
    X_flat = dataset.flatten()
    res = np.asarray(encode_ds(X_flat[::2]) + (encode_ds(X_flat[1::2])<<16), dtype=np.uint32)              
    return res

def decode(yi):
    return yi * 2**-10

def decode_pair(yab):
    ya = (yab & 0x0000ffff) * 2**-10
    ya = ya if ya < 32 else ya - 64
    yb = (yab & 0xffff0000) * 2**-26
    yb = yb if yb < 32 else yb - 64
    return ya, yb

def decode_whole_ds(dataset):
    y_13=dataset[:,:2]&0xffff
    y_24=(dataset[:,:2]&0xffff0000)>>16
    y_5 =dataset[:,2]&0xffff
    res=np.column_stack((y_13[:,0],y_24[:,0],y_13[:,1],y_24[:,1],y_5))
    return res * 2**-10

def get_output(mmio):
    y = np.zeros(6)
    for i in range(3):
        yi = decode_pair(mmio.read(4 * i))
        y[2*i], y[2*i+1] = yi[0], yi[1]
    return y[:5]

def print_dt(timea, timeb, N):
    dt = (timeb - timea) 
    rate = N / dt
    print("Classified {} samples in {} seconds ({} inferences / s)".format(N, dt, rate))
    print("----------------------------------------------------------------------------")
    print("{} s/inference ".format(1/rate))
    
def print_all_dt(timea, timeb, dts, dtr, N):
    dt  = (timeb - timea)
    rate = N / dt
    print("Send {} samples took {} seconds".format(N, dts))
    print("----------------------------------------------------------------------------")
    print("Recieve {} samples took {} seconds".format(N, dtr))
    print("----------------------------------------------------------------------------")
    print("Classified {} samples in {} seconds ({} inferences / s)".format(N, dt, rate))
    print("----------------------------------------------------------------------------")
    print("{} s/inference ".format(1/rate))

In [9]:
%timeit (encode_whole_ds(X))

100 loops, best of 3: 2.3 ms per loop


In [132]:
y = []
timea = time()
for Xi in X:
    for i in range(8):
        xab = encode_pair(Xi[2*i], Xi[2*i+1])
        in_mmio.write(4 * i, xab)
    y.append(get_output(ou_mmio))
timeb = time()

In [133]:
print_dt(timea, timeb, len(X))

Classified 1000 samples in 2.2413125038146973 seconds (446.1671446074599 inferences / s)
----------------------------------------------------------------------------
0.002241312503814697 s/inference 


In [103]:
y_enc = []
timea = time()
X_enc = encode_whole_ds(X)
for i in range(int(X_enc.shape[0]/8)):
    for j in range(8):
        in_mmio.write(4 * j, int(X_enc[i*8+j]))
    for k in range(3):
        y_temp.append(ou_mmio.read(4 * k))
    y_enc.append(y_temp)
    y_temp=[]
y = decode_whole_ds(np.asarray(y_enc))
timeb = time()

AttributeError: 'numpy.ndarray' object has no attribute 'append'

In [ ]:
print_dt(timea, timeb, len(X))

In [24]:
y_test = np.load('./y_test.npy')

In [25]:
print("Running on the board:")
for i in range(5):
    print(y[i])
print("Test output:")
for i in range(5):
    print(y_test[i])

Running on the board:
[0.61523438 0.24023438 0.01464844 0.09375    0.06445312]
[0.05761719 0.66601562 0.00976562 0.16796875 0.13085938]
[0.08789062 0.08789062 0.84179688 0.00292969 0.01464844]
[0.01464844 0.02636719 1.         0.00195312 0.00683594]
[0.03613281 0.01855469 0.94042969 0.00097656 0.00683594]
Test output:
[0. 1. 0. 0. 0.]
[0. 1. 0. 0. 0.]
[0. 0. 1. 0. 0.]
[0. 0. 1. 0. 0.]
[0. 0. 1. 0. 0.]


In [149]:
X = np.load('./X_test.npy').astype(np.float32)
y = []
timea = time()
time0 = time()
for iXi, Xi in enumerate(X):
    for i in range(8):
        xab = encode_pair(Xi[2*i], Xi[2*i+1])
        in_mmio.write(4 * i, xab)
    y.append(get_output(ou_mmio))
    if iXi % 5000 == 0:
        time1 = time()
        print_dt(time0, time1, 5000)
        time0 = time()

timeb = time()
print_dt(timea, timeb, len(X))
np.save('y_pynq.npy', y)

Classified 5000 samples in 0.009197711944580078 seconds (543613.45844782 inferences / s)
----------------------------------------------------------------------------
1.8395423889160155e-06 s/inference 
Classified 5000 samples in 11.211455345153809 seconds (445.97243141687824 inferences / s)
----------------------------------------------------------------------------
0.002242291069030762 s/inference 
Classified 5000 samples in 11.223023653030396 seconds (445.5127383296497 inferences / s)
----------------------------------------------------------------------------
0.002244604730606079 s/inference 
Classified 5000 samples in 11.256320714950562 seconds (444.19487740421584 inferences / s)
----------------------------------------------------------------------------
0.002251264142990112 s/inference 
Classified 5000 samples in 11.222935676574707 seconds (445.51623069856385 inferences / s)
----------------------------------------------------------------------------
0.0022445871353149413 s/infer

In [150]:
X = np.load('./X_test.npy').astype(np.float32)

Callback_N =5000

y_enc = []
y_temp= []
y     = []
times1 = 0
timer1 = 0
timea = time()
X_enc = encode_whole_ds(X)
time0 = time()
t_enc = time() - timea
print("Encode {} samples took {} seconds".format(len(X), t_enc))
print("----------------------------------------------------------------------------")
for i in range(int(X_enc.shape[0]/8)):
    times0 = time()
    for j in range(8):
        in_mmio.write(4 * j, int(X_enc[i*8+j]))
    times1 += (time() - times0)
    timer0 = time()
    for k in range(3):
        y_temp.append(ou_mmio.read(4 * k))
    y_enc.append(y_temp)
    y_temp=[]
    #y.append(get_output(ou_mmio))
    timer1 += (time() - timer0)
    if (i % Callback_N == 0) and i>=Callback_N :
        time1 = time()
        print_all_dt(time0, time1, times1, timer1,  Callback_N)
        time0 = time()
        times1 = 0
        timer1 = 0
        
timeb = time()
y = decode_whole_ds(np.asarray(y_enc))
t_dec = time() - timeb
print("----------------------------------------------------------------------------")
print("Decode {} samples took {} seconds".format(len(X), t_dec))
print("----------------------------------------------------------------------------")
timec = time()
print_dt(timea, timec, len(X))
np.save('y_pynq.npy', y)

Encode 166000 samples took 0.47521138191223145 seconds
----------------------------------------------------------------------------
Send 5000 samples took 1.684314489364624 seconds
----------------------------------------------------------------------------
Recieve 5000 samples took 0.5174541473388672 seconds
----------------------------------------------------------------------------
Classified 5000 samples in 2.2864739894866943 seconds (2186.773181322077 inferences / s)
----------------------------------------------------------------------------
0.0004572947978973389 s/inference 
Send 5000 samples took 1.6980023384094238 seconds
----------------------------------------------------------------------------
Recieve 5000 samples took 0.5118193626403809 seconds
----------------------------------------------------------------------------
Classified 5000 samples in 2.2907631397247314 seconds (2182.67873849272 inferences / s)
------------------------------------------------------------------

Send 5000 samples took 1.7316768169403076 seconds
----------------------------------------------------------------------------
Recieve 5000 samples took 0.5140769481658936 seconds
----------------------------------------------------------------------------
Classified 5000 samples in 2.334613084793091 seconds (2141.682505151869 inferences / s)
----------------------------------------------------------------------------
0.0004669226169586182 s/inference 
Send 5000 samples took 1.7288751602172852 seconds
----------------------------------------------------------------------------
Recieve 5000 samples took 0.517364501953125 seconds
----------------------------------------------------------------------------
Classified 5000 samples in 2.3326148986816406 seconds (2143.517132993503 inferences / s)
----------------------------------------------------------------------------
0.00046652297973632807 s/inference 
Send 5000 samples took 1.7336666584014893 seconds
-----------------------------------